# DTOcean Wave Hydrodynamics + Environmental Impact Example

Note, this example assumes the Hydrodynamics and Environmental modules has been installed

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display, HTML

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [ ]:
import numpy as np

In [ ]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu, ThemeMenu
from dtocean_core.pipeline import Tree

In [ ]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [ ]:
# Bring up the logger
start_logging()

## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [ ]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
theme_menu = ThemeMenu()
pipe_tree = Tree()

## Create a new project

In [ ]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

## Set the device type

In [ ]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type"
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Wave Floating")
my_var.read(new_core, new_project)

## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [ ]:
project_menu.initiate_pipeline(new_core, new_project)

## Discover available modules and themes

In [ ]:
names_modules = module_menu.get_available(new_core, new_project)
message1 = html_list(names_modules)
HTML(message1)

In [ ]:
names_themes = theme_menu.get_available(new_core, new_project)
message2= html_list(names_themes)
HTML(message2)

## Activate a module and a Theme

Note that the order of activation is important and that we can't deactivate yet!

In [ ]:
module_name = 'Hydrodynamics'
module_menu.activate(new_core, new_project, module_name)
theme_name = 'Environmental Impact Assessment (Experimental)'
theme_menu.activate(new_core, new_project, theme_name)

## Check the status of the module and theme inputs

In [ ]:
hydro_branch = pipe_tree.get_branch(new_core, new_project, 'Hydrodynamics')
input_status = hydro_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

In [ ]:
environmental_branch = pipe_tree.get_branch(new_core, new_project, 'Environmental Impact Assessment (Experimental)')
input_status = environmental_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [ ]:
project_menu.initiate_dataflow(new_core, new_project)

## Load test data

Prepare the test data for loading. The files required can be found in the test_data directory of the source code and should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [ ]:
%run test_data/inputs_wp2_wave.py

In [ ]:
hydro_branch.read_test_data(new_core,
                            new_project,
                            "test_data/inputs_wp2_wave.pkl")

In [ ]:
%run test_data/inputs_environmental.py

In [ ]:
environmental_branch.read_test_data(new_core,
                            new_project,
                            "test_data/inputs_environmental.pkl")

## Use the interface to enter the variable value

The interface must be "set" (entering the data) and then "connected" (moving the data into the data state)

In [ ]:
plant_rated_power = 100.
variable_id = "project.rated_power"
my_var = hydro_branch.get_input_variable(new_core, new_project, variable_id)

my_var.set_raw_interface(new_core, plant_rated_power)
my_var.read(new_core, new_project)

## Check if the module can be executed

In [ ]:
can_execute = module_menu.is_executable(new_core, new_project, module_name)
display(can_execute)

In [ ]:
can_execute_theme = theme_menu.is_executable(new_core, new_project, theme_name)
display(can_execute_theme)

In [ ]:
input_status = hydro_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

In [ ]:
input_status = environmental_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [ ]:
module_menu.execute_current(new_core, new_project)

## Examine the results

Currently, there is no robustness built into the core, so the assumption is that the module executed successfully. This will have to be improved towards deployment of the final software.

Let's check the number of devices and annual output of the farm, using just information in the data object.

In [ ]:
global_eis = new_core.get_data_value(new_project, "project.global_eis")
meta = new_core.get_metadata("project.global_eis")
name = meta.title
message_one = "<p><b>{}:</b> {}</p>".format(name, global_eis)


HTML(message_one)

## Plotting some graphs

By having data objects with set formats it should be possible to create automated plot generation. However, some plots may be too complex and some special cases may need defined.

In [ ]:
annual_energy_per_dev_value = new_core.get_data_value(new_project, 
                                                     "project.annual_energy_per_device")
meta = new_core.get_metadata("project.annual_energy_per_device")

chart_values = np.array(annual_energy_per_dev_value.values())

In [ ]:
plt.bar(range(len(annual_energy_per_dev_value)),
        chart_values,
        align='center')
            
plt.xticks(range(len(annual_energy_per_dev_value)),
           annual_energy_per_dev_value.keys())
plt.title(meta.title)

plt.ylabel(meta.units[0])

plt.tight_layout()
# plt.savefig('annual_energy_per_device.png')
plt.show()

## Plotting the Layout

This may require such a special case. It is not clear is a new data type is required or just special plots associated to variable IDs.

In [ ]:
layout_value = new_core.get_data_value(new_project, "project.layout")
layout_meta = new_core.get_metadata("project.layout")

In [ ]:
x = []
y = []

for coords in layout_value.itervalues():
    
    x.append(coords.x)
    y.append(coords.y)

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1, axisbg='lightskyblue')
ax1.plot(x,y,'k+', mew=2, markersize=10)
plt.title(layout_meta.title)
plt.axis('equal')
plt.show()